In [ ]:
import os
import shutil
import re
filelocation="D:\\3D printing\Done CCR10_helmet halo infinite master cheif.gcode"
filelocation=filelocation.replace("\\","/").split('/')
location="/".join(filelocation)
shutil.copyfile(location, location.replace('.gcode','.txt'))
reqfile=location.replace('.gcode','.txt')
layer=625

## items to find 
find=[r"(?<=Layer height:\s)[\d.]+",r"(?<=LAYER_COUNT:)[\d.]+",r"(?<=TIME:)[\d.]+",r"LAYER:0",
      r"LAYER:{}".format(layer),r"(?<=Z)[\d.]+",r"(?<=E)[\d.]+",r"(?<=TIME_ELAPSED:)[\d.]+"]

def findinfo(find):
    with open(reqfile, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            match1 = re.search(find, line)
            if match1:
                return(i,line,match1.group(0))

def findinfo2(find,end):
    with open(reqfile, 'r') as file:
        lines = file.readlines()
        newlines=lines[end:]
        for i, line in enumerate(newlines):
            match1 = re.search(find, line)
            if match1:
                return(i,line,match1.group(0))
            
def belowreplacestrings(s):
    global layer, newz, newe, newtimeelapsed,layerheight
    mod1 = re.search(r"(?<=TIME_ELAPSED:)[\d.]+",s)
    mod2 = re.search(r"(?<=LAYER:)[\d.]+",s)
    mod3 = re.search(r"(?<=Z)[\d.]+",s)
    mod4 = re.search(r"(?<=E)[\d.]+",s)
    new_s=s
    
    if mod1:
        timeelapsed = round(float(mod1.group(0)) - newtimeelapsed,5)
        new_s = re.sub(mod1.group(0), str(timeelapsed), s)

    if mod2:
        layerno = round(float(mod2.group(0)) - layer)
        new_s = re.sub(mod2.group(0), str(layerno), s)

    if mod3:
        z = round(float(mod3.group(0)) - newz + (layerheight/2),2)
        new_s = re.sub(mod3.group(0), str(z), s)

    if mod4:
        e = round(float(mod4.group(0)) - newe,5)
        new_s = re.sub(mod4.group(0), str(e), s)

    return new_s

def abovereplacestrings(s):
    global layer, newz, newe, newtimeelapsed,layerheight
    mod5 = re.search(r"(?<=LAYER_COUNT:)[\d.]+",s)
    mod6 = re.search(r"(?<=TIME:)[\d.]+",s)
    new_s=s
    
    if mod5:
        lcount = round(float(mod5.group(0)) - layer,2)
        new_s = re.sub(mod5.group(0), str(lcount), s)

    if mod6:
        newtime = round(float(mod6.group(0)) - newtimeelapsed)
        new_s = re.sub(mod6.group(0), str(newtime), s)

    return new_s


layerheight=float(findinfo(find[0])[2])
layercount=float(findinfo(find[1])[2])
time=float(findinfo(find[2])[2])
start=findinfo(find[3])[0]
end=findinfo(find[4])[0]

newz = float(findinfo2(find[5],end)[2])
newe = float(findinfo2(find[6],end)[2])
newtimeelapsed = float(findinfo2(find[7],end)[2])

# print(layerheight,layercount,time,start,end,newz,newe,newtimeelapsed)

with open(reqfile, 'r') as file:
    lines = file.readlines()
    startlines=lines[:start]
    endlines=lines[end:]
    for i, line in enumerate(startlines):
        startlines[i] = line.replace(line, abovereplacestrings(line))
    for i, line in enumerate(endlines):
        endlines[i] = line.replace(line, belowreplacestrings(line))

name=filelocation[-1].replace(".gcode", " ")
my_file=f"edited {name}{layer}.txt"
path="/".join(filelocation[:-1])
os.chdir(path)
with open(my_file, 'w') as file:
    file.seek(0)      
    file.writelines(startlines+endlines)  
    file.truncate()

base = os.path.splitext(my_file)[0]
os.remove(reqfile)
os.rename(my_file, base + '.gcode')
print("The new file edited_gcode.gcode has been generated sucessfully")